In [ ]:
!pip install rasterio
!pip install rasterstats

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.2/22.2 MB 73.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.6/56.6 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 59.9 MB/s eta 0:00:00


In [ ]:
import geopandas as gpd
import math
from rasterstats import zonal_stats
import rasterio as rio
import numpy as np

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [59]:
site_path = "/content/drive/MyDrive/Research/data/DSTM"
dsm_path = site_path + "/DSM_Clip_1.tif"
dtm_path = site_path + "/DTM_Clip_1.tif"

src_dsm = rio.open(dsm_path)
ds_dsm = src_dsm.read()
src_dsm.meta

src_dtm = rio.open(dtm_path)
ds_dtm = src_dtm.read()
src_dtm.meta

{'driver': 'GTiff',
 'dtype': 'float32',
 'nodata': -9999.0,
 'width': 578,
 'height': 563,
 'count': 1,
 'crs': CRS.from_wkt('PROJCS["WGS 84 / UTM zone 50N",GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0,AUTHORITY["EPSG","8901"]],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AUTHORITY["EPSG","4326"]],PROJECTION["Transverse_Mercator"],PARAMETER["latitude_of_origin",0],PARAMETER["central_meridian",117],PARAMETER["scale_factor",0.9996],PARAMETER["false_easting",500000],PARAMETER["false_northing",0],UNIT["metre",1,AUTHORITY["EPSG","9001"]],AXIS["Easting",EAST],AXIS["Northing",NORTH],AUTHORITY["EPSG","32650"]]'),
 'transform': Affine(0.18092510481849416, 0.0, 220837.85507008832,
        0.0, -0.18092510481851984, 81940.30412259432)}

In [60]:
ds_dsm_c = ds_dsm.copy()
ds_dtm_c = ds_dtm.copy()

ds_chm = ds_dsm_c - ds_dtm_c

In [61]:
out_meta = src_dsm.meta

chm_path = "CHM_1.tif"
with rio.open(chm_path, "w", **out_meta) as dest:
    dest.write(ds_chm)

In [62]:
crown_path = site_path + "/chm_1_crowns.gpkg"
in_crown = gpd.read_file(crown_path)
mod_in_crown = in_crown.copy()

In [63]:
crown_list = list(mod_in_crown.geometry)
mod_in_crown["height"] = 0.00
mod_in_crown["min"] = 0.00
mod_in_crown["max"] = 0.00
mod_in_crown["median"] = 0.00
mod_in_crown["majority"] = 0.00
mod_in_crown["minority"] = 0.00
mod_in_crown["std"] = 0.00
mod_in_crown["category"] = ""

for c_id, crown_poly in enumerate(crown_list):
    zstat_data = zonal_stats(crown_poly, chm_path, stats="max median std")
    print(zstat_data)
    tree_height = zstat_data[0]["max"]
    th_median = zstat_data[0]["median"]
    th_std = zstat_data[0]["std"]
    mod_in_crown.loc[c_id, "height"] = tree_height
    mod_in_crown.loc[c_id, "median"] = th_median
    mod_in_crown.loc[c_id, "std"] = th_std

    if th_median < 2 or th_std < 2:
        mod_in_crown.loc[c_id, "category"] = "non-tree"
    else:
        mod_in_crown.loc[c_id, "category"] = "tree"


mod_in_crown.to_file("zonal_1" + ".gpkg")

[{'max': 39.453887939453125, 'std': 2.97986848687514, 'median': 34.525115966796875}]
[{'max': 52.98759460449219, 'std': 7.7130964947334375, 'median': 32.984405517578125}]
[{'max': 38.53791809082031, 'std': 6.9923596818403375, 'median': 26.29981231689453}]
[{'max': 6.12042236328125, 'std': 0.7901299512251102, 'median': 0.0}]
[{'max': 2.4329986572265625, 'std': 0.3025618557266585, 'median': 0.0}]
[{'max': 3.9775238037109375, 'std': 0.07966836685190394, 'median': 0.0}]
[{'max': 2.3893585205078125, 'std': 0.14287698856455394, 'median': 0.0}]


# Fix Code

In [ ]:
def chm_process(dsm_path, dtm_path):
    src_dsm = rio.open(dsm_path)
    src_dtm = rio.open(dtm_path)

    ds_dsm = src_dsm.read()
    ds_dtm = src_dtm.read()

    ds_dsm_c = ds_dsm.copy()
    ds_dtm_c = ds_dtm.copy()

    ds_chm = ds_dsm_c - ds_dtm_c

    out_meta = src_dsm.meta

    chm_path = site_path + "/CHM_42.tif"
    with rio.open(chm_path, "w", **out_meta) as dest:
        dest.write(ds_chm)
    return chm_path

### final code

In [ ]:
from numpy import ndarray

def crown_zonalstats(crown_path, chm_path, clean=True):
    if type(crown_path) == ndarray:
        in_crown = crown_path
    else:
        in_crown = gpd.read_file(crown_path)
    # in_crown = gpd.read_file(crown_path)
    mod_in_crown = in_crown.copy()
    crown_list = list(mod_in_crown.geometry)
    mod_in_crown["height"] = 0.00

    for c_id, crown_poly in enumerate(crown_list):
        zstat_data = zonal_stats(crown_poly, chm_path, stats="max")
        tree_height = zstat_data[0]["max"]
        mod_in_crown.loc[c_id, "height"] = tree_height

    if clean:
        mod_in_crown = mod_in_crown[mod_in_crown["height"] >= 5]

    mod_in_crown.to_file("zonal_cleaned" + ".gpkg")

    return mod_in_crown

In [ ]:
from rasterio.io import MemoryFile

def zonal_cleaning(dsm_path, dtm_path, crown_path, to_file=False, clean_crown=True):
    # CHM generation from DSM and DTM
    src_dsm = rio.open(dsm_path)
    src_dtm = rio.open(dtm_path)

    ds_dsm = src_dsm.read()
    ds_dtm = src_dtm.read()

    ds_dsm_c = ds_dsm.copy()
    ds_dtm_c = ds_dtm.copy()

    ds_chm = ds_dsm_c - ds_dtm_c

    # If user want CHM output
    if to_file:
        out_meta = src_dsm.meta

        chm_path = site_path + "/CHM_42.tif"
        with rio.open(chm_path, "w", **out_meta) as dest:
            dest.write(ds_chm)

        return crown_zonalstats(crown_path, chm_path, clean_crown)

    # Otherwise, chm only as temp file
    else:
        with MemoryFile() as temp_file:
          with temp_file.open(**src_dsm.meta) as dataset:
              dataset.write(ds_chm)

          return crown_zonalstats(crown_path, temp_file, clean_crown)

In [ ]:
crown_path = site_path + "/chm_4_crowns.gpkg"
cleaned = zonal_cleaning(dsm_path, dtm_path, crown_path, to_file=False, clean_crown=False)

In [ ]:
cleaned.head(10)

,id,geometry,height
0,NaN,"MULTIPOLYGON (((220583.762 82278.104, 220583.3...",27.830887
1,NaN,"MULTIPOLYGON (((220559.835 82242.788, 220559.3...",5.440659
2,NaN,"MULTIPOLYGON (((220618.134 82257.268, 220617.5...",0.000000
3,NaN,"MULTIPOLYGON (((220582.423 82259.634, 220582.1...",0.000000
4,NaN,"MULTIPOLYGON (((220626.264 82279.839, 220625.8...",7.232574
5,NaN,"MULTIPOLYGON (((220616.935 82223.844, 220616.6...",28.164322
